Ноутбук по курсовой. Задача: классификация текстов на три класса.
Подходы: NB и нейросети.

Обозначения в корпусе: 1 - подтверждающие, 2 - опровергающие, 3 - упоминающие, 4 - неопределённые.
Есть еще 0

In [65]:
import pandas as pd

data = pd.read_csv('verum_2017-12-3.csv', header=0, sep=',')
data = data[['id', 'text', 'classification']]
#Решила пока просто удалить неопределенные
data = data[(data.classification != 0)]
data = data[(data.classification != 4)]

#Посмотрим по сколько сэмплов каждого класса
print(len(data[(data.classification == 1)].axes[0]))
print(len(data[(data.classification == 2)].axes[0]))
print(len(data[(data.classification == 3)].axes[0]))

2404
433
42769


В данных мало примеров классов 1 и 2 и очень много 3. Что с этим можно сделать?

In [68]:
from sklearn import metrics

def print_metrics(y_true, y_pred):
    """
    Функция, печатающая необходимые метрики.
    y_true - реальные метки классов
    y_pred - предсказанные метки классов
    """
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    print(confusion_matrix)
    print('Precision neg vs all:', round(confusion_matrix[1][1]/sum(confusion_matrix[1]), 2))
    print('Recall neg vs all:', round(confusion_matrix[1][1]/sum(list(zip(*confusion_matrix[:]))[1]), 2))
          
    print('Precision neg vs pos:', round(confusion_matrix[1][1]/sum(confusion_matrix[1][:2]), 2))
    print('Recall neg vs pos:', round(confusion_matrix[1][1]/sum(list(zip(*confusion_matrix[:2]))[1]), 2))

In [83]:
from sklearn.model_selection import train_test_split

#Разобьем на train test 90/10. С учетом неравномерности классов использовала стратификацию.
X_train, X_test, y_train, y_test = train_test_split(data.text.astype('str'), data.classification, test_size=0.1, random_state=42, stratify=data.classification)

#Разобьем train на train и validation для установки параметра alpha
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)
    

In [84]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Посмотрим на параметр от 0 до 0.1
for a in [round(0.01*x, 2) for x in range(0, 11)]:
    print ('alpha =', a)
    #Фичи - количества слов + удаляем стоп-слова все
    text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                         ('clf', MultinomialNB(alpha=a, class_prior=None, fit_prior=True)),])
    
    text_clf.fit(X_train_, y_train_)
    predicted = text_clf.predict(X_val)
    
    print_metrics(y_val, predicted)

alpha = 0.0


C:\Users\novon\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


[[  11    5  200]
 [   2    1   36]
 [  69    7 3774]]
Precision neg vs all: 0.03
Recall neg vs all: 0.08
Precision neg vs pos: 0.33
Recall neg vs pos: 0.17
alpha = 0.01
[[  34    7  175]
 [   8    1   30]
 [ 180   25 3645]]
Precision neg vs all: 0.03
Recall neg vs all: 0.03
Precision neg vs pos: 0.11
Recall neg vs pos: 0.12
alpha = 0.02
[[  36    7  173]
 [   9    1   29]
 [ 220   27 3603]]
Precision neg vs all: 0.03
Recall neg vs all: 0.03
Precision neg vs pos: 0.1
Recall neg vs pos: 0.12
alpha = 0.03
[[  39    7  170]
 [   9    1   29]
 [ 232   29 3589]]
Precision neg vs all: 0.03
Recall neg vs all: 0.03
Precision neg vs pos: 0.1
Recall neg vs pos: 0.12
alpha = 0.04
[[  42    7  167]
 [   9    1   29]
 [ 244   29 3577]]
Precision neg vs all: 0.03
Recall neg vs all: 0.03
Precision neg vs pos: 0.1
Recall neg vs pos: 0.12
alpha = 0.05
[[  42    7  167]
 [   9    1   29]
 [ 251   28 3571]]
Precision neg vs all: 0.03
Recall neg vs all: 0.03
Precision neg vs pos: 0.1
Recall neg vs pos: 0.

Сначала посмотрела alpha от 0 до 1 с шагом 0.1. Стало понятно, что при меньшем лучше, посмотрела от 0 до 0.1 с шагом 0.01. При alpha=0 лучше интересующие метрики, но все равно очень грустные.  

In [7]:
#Проверим метрики еще на test
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('clf', MultinomialNB(alpha=0, class_prior=None, fit_prior=True)),])

text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
print_metrics(y_test, predicted)

C:\Users\novon\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


[[  15    2  224]
 [   7    2   34]
 [  80    2 4195]]
Precision neg vs all: 0.05
Recall neg vs all: 0.33
Precision neg vs pos: 0.22
Recall neg vs pos: 0.5


Веселее не стало :)

Начала делать подход с нейросетями и решила подробнее посмотреть что там за тексты. Оказалось, что там очень много слов, не несущих существенной информации (типа обозначения в формулах и цифры, есть еще всякие последовательности нуклеотидов типа 'aagctaact'). Решила сделать доп. обработку, которая пришла в голову: удалить слова с цифрами или греческими символами, стоп-слова(кроме отрицаний пока на случай 'not confirmed') и провести стемминг.

In [145]:
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords

#Удалим слова с цифрами и символами греческого алфавита.

def my_preprocessing(data):
    """
    Функция, которая удаляет из текстов стоп-слова, символы из формул (из очевидного с цифрами и буквами греческого алфавита).
    Производит стемминг слов.
    
    data - последовательность документов(текстов)
    return texts - список обработанных текстов
    """
    #Множество греческих символов
    GREEK_ALPHABET = {u'\u0391', u'\u0392', u'\u0393', u'\u0394', u'\u0395', u'\u0396', u'\u0397', u'\u0398', u'\u0399',
                      u'\u039A', u'\u039B', u'\u039C', u'\u039D', u'\u039E', u'\u039F', u'\u03A0', u'\u03A1', u'\u03A3',
                      u'\u03A4', u'\u03A5', u'\u03A6', u'\u03A7', u'\u03A8', u'\u03A9', u'\u03B1', u'\u03B2', u'\u03B3',
                      u'\u03B4', u'\u03B5', u'\u03B6', u'\u03B7', u'\u03B8', u'\u03B9', u'\u03BA', u'\u03BB', u'\u03BC',
                      u'\u03BD', u'\u03BE', u'\u03BF', u'\u03C0', u'\u03C1', u'\u03C3', u'\u03C4', u'\u03C5', u'\u03C6',
                      u'\u03C7', u'\u03C8', u'\u03C9'}
    
    #Множество цифр
    DIGITS = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}
    
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    allow_stop_words = {"not", "doesn't", "isn't", "aren't", "can't", "couldn't", "don't", "hadn't", "hasn't", "haven't", "no" }
    
    texts = []
    for i, t in enumerate(data):
        if isinstance(t, float):
            t = str(t)
        texts.append(' '.join([stemmer.stem(w.lower()) for w in t.split(' ') 
                               if not w in stop_words - allow_stop_words 
                               and not bool(set(w) & (GREEK_ALPHABET | DIGITS))]))
    return texts


In [149]:
#Попробуем теперь опять сделать NB с такой предобработкой.
documents = my_preprocessing(data['text'])


45606


In [150]:
from sklearn.model_selection import train_test_split

#Разобьем на train test 90/10. С учетом неравномерности классов использовала стратификацию.
X_train, X_test, y_train, y_test = train_test_split(documents, data.classification, test_size=0.1, random_state=42, stratify=data.classification)

#Разобьем train на train и validation для установки параметра alpha
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [151]:
#Варьируем alpha от 0 до 0.1 с шагом 0.01
for a in [round(0.01*x, 2) for x in range(0, 11)]:
    print ('alpha =', a)
 
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('clf', MultinomialNB(alpha=a, class_prior=None, fit_prior=True)),])
    
    text_clf.fit(X_train_, y_train_)
    predicted = text_clf.predict(X_val)
    
    print_metrics(y_val, predicted)

alpha = 0.0


C:\Users\novon\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


[[  20    5  191]
 [   4    1   34]
 [ 116    6 3728]]
Precision neg vs all: 0.03
Recall neg vs all: 0.08
Precision neg vs pos: 0.2
Recall neg vs pos: 0.17
alpha = 0.01
[[  53    9  154]
 [   8    1   30]
 [ 271   28 3551]]
Precision neg vs all: 0.03
Recall neg vs all: 0.03
Precision neg vs pos: 0.11
Recall neg vs pos: 0.1
alpha = 0.02
[[  56    9  151]
 [   9    1   29]
 [ 305   38 3507]]
Precision neg vs all: 0.03
Recall neg vs all: 0.02
Precision neg vs pos: 0.1
Recall neg vs pos: 0.1
alpha = 0.03
[[  62    9  145]
 [  10    2   27]
 [ 328   43 3479]]
Precision neg vs all: 0.05
Recall neg vs all: 0.04
Precision neg vs pos: 0.17
Recall neg vs pos: 0.18
alpha = 0.04
[[  63    9  144]
 [  10    2   27]
 [ 346   41 3463]]
Precision neg vs all: 0.05
Recall neg vs all: 0.04
Precision neg vs pos: 0.17
Recall neg vs pos: 0.18
alpha = 0.05
[[  63    9  144]
 [  10    2   27]
 [ 358   42 3450]]
Precision neg vs all: 0.05
Recall neg vs all: 0.04
Precision neg vs pos: 0.17
Recall neg vs pos: 0.

In [153]:
#Проверим метрики еще на test
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('clf', MultinomialNB(alpha=0, class_prior=None, fit_prior=True)),])

text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
print_metrics(y_test, predicted)

C:\Users\novon\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


[[  18    4  219]
 [   7    2   34]
 [ 132    4 4141]]
Precision neg vs all: 0.05
Recall neg vs all: 0.2
Precision neg vs pos: 0.22
Recall neg vs pos: 0.33


Метрики стали даже похуже. Но думаю, что они более достоверны.
В любом случае это все выглядит не очень, хотя если смотреть на accuracy, то она выходит в районе 0.9 просто за счет количества нейтральных примеров. 

In [154]:
"""
Попробуем что-нибудь сделать с нейронками.
Обучим word2vec на нашем корпусе и потом используем полученные представления в качестве фичей документов 
(напр. среднее по всем векторам слов в документе). И попробуем для начала использовать это в какой-нибудь простенькой NN модели.

Вообще говоря, наш корпус не очень большой и было бы круче обучить word2vec на большем датасете по соотв. теме. 
Но где бы такой взять? 
"""


import gensim
import nltk
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

#documents - обработанные документы
#Нужно разбить тексты по предложениям. Предложения на слова.
sentences = []
for document in documents:
    doc_sentences = sent_tokenize(document)
    for sentence in doc_sentences:
        sentences.append([w for w in word_tokenize(sentence) if w not in string.punctuation])


In [155]:
#Посмотрим сколько у нас слов
print(sum([len(s) for s in sentences]))

4386838


In [170]:
#Скормим наш корпус в gensim.Word2Vec и обучим модель
model = gensim.models.Word2Vec(
    sentences,
    #Длина вектора
    size=100,
    #Сколько слов смотрим рядом
    window=2,
    #Минимальное количество токена в корпусе, если меньше, то выкидывается 
    min_count=2,
    #Модель (skip-gram или cbow). Использую skip-gram, пишут, что лучше подходит для небольших корпусов.
    sg = 1
)

model.train(sentences, total_examples=len(sentences), epochs=10)


(41776021, 43868380)

In [171]:
model.save('window2_mincount_2.w2v')

In [ ]:
w2v = dict(zip(model.wv.index2word, model.wv.vectors))
print(w2v)

In [172]:
# Посмотрим на что-нибудь внутри модели. Какие слова похожи на confirm, например
w1 = ['confirm']
model.wv.most_similar(positive=w1)

[('verifi', 0.8220750093460083),
 ('corrobor', 0.6523631811141968),
 ('demonstr', 0.647998571395874),
 ('confirmation', 0.6387323141098022),
 ('support', 0.6176462173461914),
 ('reveal', 0.6165704727172852),
 ('examin', 0.6141142845153809),
 ('suggest', 0.6060745120048523),
 ('indicate', 0.6044885516166687),
 ('demonstrate', 0.6010904908180237)]

Выглядит вполне правдоподобно.
Попрубуем использовать это каким-то образом как фичи.
Например, возьмем для каждого документа средний вектор по всем словам, которые в нем содержатся

In [173]:
#Длина вектора
size = 100
#Функция вычисляющая среднее по документу
def get_w2v_mean(document):
    vec = np.zeros(size)
    count = 0.
    for word in document:
        try:
            vec += model.wv[word]
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [186]:
#Сделаем из списка документов список списков слов в документах
documents_words = []
for document in documents:
    documents_words.append([w for w in word_tokenize(document) if w not in string.punctuation])

In [188]:

#Разобьем на train test 90/10. С учетом неравномерности классов использовала стратификацию.
X_train, X_test, y_train, y_test = train_test_split(documents, data.classification, test_size=0.1, random_state=42, stratify=data.classification)

X_train_vec = []
for document in X_train:
    X_train_vec.append(get_w2v_mean(document))
    
X_test_vec = []
for document in X_test:
    X_test_vec.append(get_w2v_mean(document))
    

In [246]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

#Номера классов от 0 до 2
y_train_binary = to_categorical([y - 1 for y in y_train])

#Простенькая модель
nn_model = Sequential()
nn_model.add(Dense(256, activation='relu', input_dim=100))
nn_model.add(Dropout(0.25))
nn_model.add(Dense(256, activation='relu'))
nn_model.add(Dense(3, activation='sigmoid'))
nn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

#Поставила более редко встречающимся классам более высокие веса, иначе все классифицировалось как нейтральное
class_weight = {0: 20.,
                1: 100.,
                2: 1.}

nn_model.fit(np.array(X_train_vec), y_train_binary, batch_size=128, epochs=200, verbose=2, validation_split=0.2, class_weight=class_weight)

Train on 32836 samples, validate on 8209 samples
Epoch 1/200
 - 3s - loss: 3.2905 - categorical_accuracy: 0.0929 - val_loss: 3.0650 - val_categorical_accuracy: 0.0079
Epoch 2/200
 - 2s - loss: 3.2768 - categorical_accuracy: 0.0383 - val_loss: 3.0550 - val_categorical_accuracy: 0.0525
Epoch 3/200
 - 2s - loss: 3.2751 - categorical_accuracy: 0.0648 - val_loss: 3.0633 - val_categorical_accuracy: 0.0525
Epoch 4/200
 - 2s - loss: 3.2744 - categorical_accuracy: 0.0528 - val_loss: 3.0558 - val_categorical_accuracy: 0.0525
Epoch 5/200
 - 2s - loss: 3.2735 - categorical_accuracy: 0.0527 - val_loss: 3.0536 - val_categorical_accuracy: 0.0525
Epoch 6/200
 - 2s - loss: 3.2733 - categorical_accuracy: 0.0527 - val_loss: 3.0525 - val_categorical_accuracy: 0.0525
Epoch 7/200
 - 2s - loss: 3.2728 - categorical_accuracy: 0.0527 - val_loss: 3.0534 - val_categorical_accuracy: 0.0525
Epoch 8/200
 - 2s - loss: 3.2722 - categorical_accuracy: 0.0527 - val_loss: 3.0540 - val_categorical_accuracy: 0.0525
Epoch 9

Epoch 70/200
 - 2s - loss: 3.2373 - categorical_accuracy: 0.3884 - val_loss: 3.0195 - val_categorical_accuracy: 0.3664
Epoch 71/200
 - 2s - loss: 3.2313 - categorical_accuracy: 0.4377 - val_loss: 3.0098 - val_categorical_accuracy: 0.6286
Epoch 72/200
 - 2s - loss: 3.2351 - categorical_accuracy: 0.4127 - val_loss: 3.0216 - val_categorical_accuracy: 0.3415
Epoch 73/200
 - 2s - loss: 3.2310 - categorical_accuracy: 0.4273 - val_loss: 3.0359 - val_categorical_accuracy: 0.7286
Epoch 74/200
 - 2s - loss: 3.2292 - categorical_accuracy: 0.3682 - val_loss: 3.0128 - val_categorical_accuracy: 0.5805
Epoch 75/200
 - 2s - loss: 3.2329 - categorical_accuracy: 0.4478 - val_loss: 3.0254 - val_categorical_accuracy: 0.3488
Epoch 76/200
 - 2s - loss: 3.2356 - categorical_accuracy: 0.4429 - val_loss: 3.0094 - val_categorical_accuracy: 0.5087
Epoch 77/200
 - 2s - loss: 3.2350 - categorical_accuracy: 0.4010 - val_loss: 3.0371 - val_categorical_accuracy: 0.2505
Epoch 78/200
 - 2s - loss: 3.2314 - categorical_

Epoch 139/200
 - 2s - loss: 3.2171 - categorical_accuracy: 0.4160 - val_loss: 3.0241 - val_categorical_accuracy: 0.2933
Epoch 140/200
 - 2s - loss: 3.2265 - categorical_accuracy: 0.3729 - val_loss: 3.0040 - val_categorical_accuracy: 0.3601
Epoch 141/200
 - 2s - loss: 3.2155 - categorical_accuracy: 0.4029 - val_loss: 3.0320 - val_categorical_accuracy: 0.2835
Epoch 142/200
 - 2s - loss: 3.2139 - categorical_accuracy: 0.4250 - val_loss: 3.0565 - val_categorical_accuracy: 0.1486
Epoch 143/200
 - 2s - loss: 3.2163 - categorical_accuracy: 0.4321 - val_loss: 3.0169 - val_categorical_accuracy: 0.3255
Epoch 144/200
 - 2s - loss: 3.2188 - categorical_accuracy: 0.3739 - val_loss: 3.0334 - val_categorical_accuracy: 0.1882
Epoch 145/200
 - 2s - loss: 3.2218 - categorical_accuracy: 0.4036 - val_loss: 2.9980 - val_categorical_accuracy: 0.4817
Epoch 146/200
 - 2s - loss: 3.2160 - categorical_accuracy: 0.4121 - val_loss: 3.0011 - val_categorical_accuracy: 0.5986
Epoch 147/200
 - 2s - loss: 3.2253 - cat

In [247]:
p = nn_model.predict(np.array(X_test_vec))
y_pred = np.argmax(p, axis=-1)

In [248]:
 print_metrics([y-1 for y in y_test], y_pred)

[[  82   92   67]
 [  16   12   15]
 [1215 1232 1830]]
Precision neg vs all: 0.28
Recall neg vs all: 0.01
Precision neg vs pos: 0.43
Recall neg vs pos: 0.12


Как-то так.. Видно, что модель не очень, но дает надежду, что если покопаться в архитектуре, параметрах и пр., может дать хороший результат. Только вот в какую сторону двигаться, я не знаю.

Плюс в нейросетях я разбираюсь так себе пока что и ставлю параметры из чувства прекрасного и гугла :)

Можно подумать о другой архитектуре (какой?), использовать другой loss (какой?), другие фичи(какие?)

Мне кажется, что основная сложность в том, что мало примеров pos и neg.